In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score

In [7]:
# классификаторы

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [8]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
y_true = pd.read_csv('titanic_test.csv')["Survived"]

In [9]:
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [10]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [11]:
# количества ячеек пропущенными значениями по столбцам 

def sum_nan(x):
    return len([1 for a in x if a==True])

df.isnull().apply(sum_nan)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
df[['Cabin', 'Embarked']] = df[['Cabin', 'Embarked']].fillna('U')

In [15]:
male_median, female_median = df.groupby('Sex')["Age"].agg('median')
print(f'For males: {np.round(male_median)},\nfor females: {np.round(female_median)}.')

For males: 27.0,
for females: 29.0.


In [16]:
def age_nan(x, y, a, b):
    ans = []
    for i in range(len(x)):
        if y[i] >- 1:
            ans.append(y[i])
        else:
            if x[i] == 'male':
                ans.append(a)
            else:
                ans.append(b)
    return ans
    
df["Age"] = age_nan(df["Sex"], df["Age"], male_median, female_median)

In [17]:
df.isnull().apply(sum_nan)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [18]:
df["Sex"] = df["Sex"].map({'male':0, 'female':1})
df["Embarked"] = df["Embarked"].map({'U':0, 'S':1, 'C':2, 'Q':3,})

del df['PassengerId'], df['Name'], df['Ticket'], df['Cabin']
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,1
887,1,1,1,19.0,0,0,30.0000,1
888,0,3,1,29.0,1,2,23.4500,1
889,1,1,0,26.0,0,0,30.0000,2


In [19]:
df_test["Age"] = age_nan(df_test["Sex"], df_test["Age"], male_median, female_median)
df_test["Sex"] = df_test["Sex"].map({'male':0, 'female':1})
df_test["Embarked"] = df_test["Embarked"].map({'U':0, 'S':1, 'C':2, 'Q':3,})

del df_test['PassengerId'], df_test['Name'], df_test['Ticket'], df_test['Cabin']
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,3
1,3,1,47.0,1,0,7.0000,1
2,2,0,62.0,0,0,9.6875,3
3,3,0,27.0,0,0,8.6625,1
4,3,1,22.0,1,1,12.2875,1
...,...,...,...,...,...,...,...
413,3,0,27.0,0,0,8.0500,1
414,1,1,39.0,0,0,108.9000,2
415,3,0,38.5,0,0,7.2500,1
416,3,0,27.0,0,0,8.0500,1


In [20]:
y_train = df["Survived"]
del df["Survived"]

df_test.isnull().apply(sum_nan)

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        1
Embarked    0
dtype: int64

In [21]:
df_test["Fare"] = df_test["Fare"].fillna(df_test["Fare"].mean())

In [22]:
X_train, X_test = df, df_test

In [23]:
# K-Nearest-Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn_model = knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
knn_accuracy = accuracy_score(y_true, y_pred)
knn_precision = precision_score(y_true, y_pred)
knn_recall = recall_score(y_true, y_pred)

In [ ]:
# Naive Bayes classifier
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
nb_accuracy = accuracy_score(y_true, y_pred)
nb_precision = precision_score(y_true, y_pred)
nb_recall = recall_score(y_true, y_pred)

In [ ]:
# Decision Tree classifier
dt = DecisionTreeClassifier()
dt_model = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_true, y_pred)
dt_precision = precision_score(y_true, y_pred)
dt_recall = recall_score(y_true, y_pred)

In [24]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg_model = log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
log_reg_accuracy = accuracy_score(y_true, y_pred)
log_reg_precision = precision_score(y_true, y_pred)
log_reg_recall = recall_score(y_true, y_pred)

C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


В каждом из пунктов следующего теста выберите, что важнее максимизировать: точность (precision) или полноту (recall). Выпишите ответы.<br><br>

1. Вероятность того, что при определенной поломке самолета он сможет долететь до пункта назначения (1 - долетел, 0 - не долетел).<br>
2. Предсказание, представляет ли человек опасность, по анализу психического состояния (1 - представляет опасность, 0 - не представляет опасности).<br>
3. Предсказание ухода клиента (1 - клиент ушел, 0 - остался).<br>
4. Выявление рака на основе медицинских показателей (1 - болен раком, 0 - здоров).<br>
5. Предсказание летальности при наблюдаемой мутации (1 - выживание, 0 - летальный исход).<br>
6. Определение важности происшествия для экстренных служб (1 - важно, 0 - неважно).<br>
7. Окупятся ли вложения в бизнес (1 - окупятся, 0 - не окупятся).<br>

1. recall.<br>
2. precision.<br>
3. precision.<br>
4. recall.<br>
5. recall.<br>
6. precision.<br>
7. precision.<br>

Насчет Титаника: я считаю, разумнее использовать полноту (**recall**).

Полнота - метрика, представляющая информацию о том, сколько мы упустили. Очевидно, что лучше предсказать смерть пассажира и не пустить его на борт, даже если он является потенциальным выжившим, чем ошибочно спрогнозировать успешный исход для того, кто в конечном итоге умрет. Одним словом, в случае печального известного лайнера мы оперируем слишком дорогой валютой - человеческой жизнью, и права на ошибку нет.

In [ ]:
print(f'K-Nearest-Neighbors classifier: recall-{np.round(knn_recall, 2)}, precision-{np.round(knn_precision, 2)}\n\
Naive Bayes classifier: recall-{np.round(nb_recall, 2)}, precision-{np.round(nb_precision, 2)}\n\
Decision Tree classifier: recall-{np.round(dt_recall, 2)}, precision-{np.round(dt_precision, 2)}\n\
Logistic Regressoin: recall-{np.round(log_reg_recall, 2)}, precision-{np.round(log_reg_precision, 2)}')

Как видим, наибольшее значение метрика **recall** принимает при классификации методом **Naive Bayes**.